In [8]:
#simple flow
#->Environment setup by installed required libraries
#->converting the whole pdf into images
#->selecting only those images which are having table and cropping it
#->now selected cropped images will be converted into csv files
#->those csv files will be meaged into one txt
#->it will be given to the input to langflow api for futher detection


In [ ]:
#taken company->utiasset

# ***Environment Setup***

In [1]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
!pip install paddlepaddle-gpu==2.5.2
!pip install paddlepaddle-gpu==2.3.0.post110 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install pdf2image
!apt-get update
!apt-get install poppler-util
!pip install pdf2image
!apt-get update
!apt-get install poppler-utils
#!python3 -m pip install paddlepaddle-gpu
!pip install "paddleocr>=2.0.1"
!pip install protobuf==3.20.0
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
ERROR: Could not find a version that satisfies the requirement paddlepaddle-gpu==2.3.0.post110 (from versions: 2.5.1, 2.5.1.post102, 2.5.1.post112, 2.5.1.post116, 2.5.1.post117, 2.5.1.post120, 2.5.2, 2.5.2.post102, 2.5.2.post112, 2.5.2.post116, 2.5.2.post117, 2.5.2.post120, 2.6.0, 2.6.0.post112, 2.6.0.post116, 2.6.0.post117, 2.6.0.post120, 2.6.1, 2.6.1.post112, 2.6.1.post116, 2.6.1.post117, 2.6.1.post120, 2.6.2)
ERROR: No matching distribution found for paddlepaddle-gpu==2.3.0.post110
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,306 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http:/

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 99136, done.
remote: Counting objects: 100% (4127/4127), done.
remote: Compressing objects: 100% (515/515), done.
remote: Total 99136 (delta 3845), reused 3773 (delta 3612), pack-reused 95009 (from 3)
Receiving objects: 100% (99136/99136), 623.86 MiB | 17.27 MiB/s, done.
Resolving deltas: 100% (75448/75448), done.
Updating files: 100% (2024/2024), done.
--2025-02-01 04:44:05--  https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.47.176, 2409:8c04:1001:1203:0:ff:b0bb:4f27
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.47.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19145360 (18M) [application/octet-stream]
Saving to: ‘layoutparser-0.0.0-py3-none-any.whl’

layoutparser-0.0.0- 100%[===================>]  18.26M  15.7MB/s    in 1.2s    

2025-02-01 04:44:08 (15.7 MB/s) - ‘layoutparser-0.

# **Converting the whole pdf into images(page wise)**

In [3]:
from pdf2image import convert_from_path
images = convert_from_path('/content/utiasset.pdf')
!mkdir pages
for i in range(len(images)):
  images[i].save('pages/page'+str(i)+'.jpg', 'JPEG')

Helper code


In [65]:
# import os
# import glob

# # Path to the directory containing images
# image_folder = "/content/pages"  # Change this to your directory

# # File extensions to delete
# image_extensions = ["*.jpg"]

# # Delete all images except those starting with "page"
# for ext in image_extensions:
#     for file in glob.glob(os.path.join(image_folder, ext)):
#         filename = os.path.basename(file)
#         # if not filename.startswith("page"):  # Skip files starting with "page"
#         os.remove(file)
#         print(f"Deleted: {file}")
#         # else:
#         #     print(f"Skipped: {file}")

# print("✅ Cleanup complete!")


Deleted: /content/pages/page2.jpg
Deleted: /content/pages/page1.jpg
Deleted: /content/pages/page0.jpg
Deleted: /content/pages/page4.jpg
Deleted: /content/pages/page9.jpg
Deleted: /content/pages/cropped_page9_table.jpg
Deleted: /content/pages/page3.jpg
Deleted: /content/pages/cropped_page4_table.jpg
Deleted: /content/pages/cropped_page5_table.jpg
Deleted: /content/pages/page8.jpg
Deleted: /content/pages/page6.jpg
Deleted: /content/pages/cropped_page3_table.jpg
Deleted: /content/pages/page5.jpg
Deleted: /content/pages/cropped_page8_table.jpg
Deleted: /content/pages/page7.jpg
✅ Cleanup complete!


# **Taking only those images which are having table and cropping that**

In [4]:
import cv2
import layoutparser as lp
import os

# Function to crop images dynamically and ensure nothing is cut off
def crop_images_with_tables(directory_path):
    # Load the model once for detecting tables
    model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                          threshold=0.1,  # Lower threshold for detecting more tables
                                          label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
                                          enforce_cpu=False,
                                          enable_mkldnn=True)

    # Iterate over all pages in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpg"):  # Process only .jpg files
            image_path = os.path.join(directory_path, filename)
            image = cv2.imread(image_path)

            # Check if the image is loaded correctly
            if image is None:
                print(f"Error loading image {filename}")
                continue

            height, width, _ = image.shape  # Get image dimensions

            # Detect layout in the image (detect tables, etc.)
            layout = model.detect(image)

            # Initialize bounding box for cropping
            min_x, min_y, max_x, max_y = width, height, 0, 0

            # Iterate over detected layout elements
            for l in layout:
                if l.type == 'Table':
                    x_1 = max(0, int(l.block.x_1))  # Ensure within bounds
                    y_1 = max(0, int(l.block.y_1))
                    x_2 = min(width, int(l.block.x_2))
                    y_2 = min(height, int(l.block.y_2))

                    # Calculate an adaptive margin (5% of table size)
                    margin_x = int((x_2 - x_1) * 0.05)
                    margin_y = int((y_2 - y_1) * 0.05)

                    min_x = min(min_x, x_1 - margin_x)
                    min_y = min(min_y, y_1 - margin_y)
                    max_x = max(max_x, x_2 + margin_x)
                    max_y = max(max_y, y_2 + margin_y)

            # Ensure valid cropping area
            if min_x < max_x and min_y < max_y:
                min_x = max(0, min_x)
                min_y = max(0, min_y)
                max_x = min(width, max_x)
                max_y = min(height, max_y)

                cropped_im = image[min_y:max_y, min_x:max_x]

                # Check if the cropped image is valid
                if cropped_im.size == 0:
                    print(f"The cropped image is empty for {filename}!")
                else:
                    # Save the cropped image
                    cropped_image_filename = f"cropped_{filename.split('.')[0]}_table.jpg"
                    cv2.imwrite(os.path.join(directory_path, cropped_image_filename), cropped_im)
                    print(f"Table cropped from {filename} and saved as {cropped_image_filename}")
            else:
                print(f"No valid table detected in {filename}, no crop performed.")

# Specify the directory containing the images of pages
directory_path = "/content/pages"
crop_images_with_tables(directory_path)




download https://paddle-model-ecology.bj.bcebos.com/model/layout-parser/ppyolov2_r50vd_dcn_365e_publaynet.tar to /root/.paddledet/inference_model/ppyolov2_r50vd_dcn_365e_publaynet/ppyolov2_r50vd_dcn_365e_publaynet_infer/ppyolov2_r50vd_dcn_365e_publaynet.tar


100%|██████████| 221M/221M [00:05<00:00, 38.9MiB/s]


Table cropped from page2.jpg and saved as cropped_page2_table.jpg
No valid table detected in page1.jpg, no crop performed.
No valid table detected in page0.jpg, no crop performed.
Table cropped from page4.jpg and saved as cropped_page4_table.jpg
No valid table detected in page3.jpg, no crop performed.
No valid table detected in page8.jpg, no crop performed.
Table cropped from page6.jpg and saved as cropped_page6_table.jpg
Table cropped from page5.jpg and saved as cropped_page5_table.jpg
Table cropped from page7.jpg and saved as cropped_page7_table.jpg


# **Converting those selected cropped images to csv**

In [5]:
from paddleocr import PaddleOCR
import tensorflow as tf
import cv2
import os
import numpy as np
import pandas as pd

# Initialize PaddleOCR with improved settings
ocr = PaddleOCR(lang='en', rec_algorithm='CRNN')

# Function to process each cropped image and generate a structured CSV
def process_images_and_generate_csv(directory_path, output_directory):
    # Ensure output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Iterate over all cropped images in the directory
    for filename in os.listdir(directory_path):
        if filename.startswith("cropped_") and filename.endswith(".jpg"):  # Process only cropped images
            cropped_image_path = os.path.join(directory_path, filename)
            image_cv = cv2.imread(cropped_image_path)

            # Check if the image is loaded correctly
            if image_cv is None:
                print(f"Error loading cropped image {filename}")
                continue

            # Perform OCR on the cropped image
            output = ocr.ocr(cropped_image_path, cls=True)

            if not output or not output[0]:  # Ensure OCR detected something
                print(f"No text detected in {filename}, skipping...")
                continue

            # Extract bounding boxes, text, and confidence scores
            boxes = [line[0] for line in output[0]]
            texts = [line[1][0] for line in output[0]]
            probabilities = [line[1][1] for line in output[0]]

            # Separate horizontal and vertical bounding boxes
            horiz_boxes, vert_boxes = [], []
            for box in boxes:
                x_h, y_h = 0, int(box[0][1])  # Full width, row-based
                width_h, height_h = image_cv.shape[1], int(box[2][1] - box[0][1])
                horiz_boxes.append([x_h, y_h, x_h + width_h, y_h + height_h])

                x_v, y_v = int(box[0][0]), 0  # Full height, column-based
                width_v, height_v = int(box[2][0] - box[0][0]), image_cv.shape[0]
                vert_boxes.append([x_v, y_v, x_v + width_v, y_v + height_v])

            # Convert to tensor format for Non-Maximum Suppression (NMS)
            horiz_boxes_tensor = tf.convert_to_tensor(horiz_boxes, dtype=tf.float32)
            vert_boxes_tensor = tf.convert_to_tensor(vert_boxes, dtype=tf.float32)
            probabilities_tensor = tf.convert_to_tensor(probabilities, dtype=tf.float32)

            # Apply NMS for horizontal (rows) and vertical (columns)
            horiz_out = tf.image.non_max_suppression(horiz_boxes_tensor, probabilities_tensor, max_output_size=500, iou_threshold=0.2)
            vert_out = tf.image.non_max_suppression(vert_boxes_tensor, probabilities_tensor, max_output_size=500, iou_threshold=0.2)

            # Sort detected lines and columns properly
            horiz_lines = np.sort(np.array(horiz_out))
            vert_lines = np.sort(np.array(vert_out))

            # Prepare an empty matrix for structured CSV
            out_array = [["" for _ in range(len(vert_lines))] for _ in range(len(horiz_lines))]

            unordered_boxes = [vert_boxes[i][0] for i in vert_lines]
            ordered_boxes = np.argsort(unordered_boxes)

            # Helper function: Calculate Intersection over Union (IoU)
            def iou(box1, box2):
                x1 = max(box1[0], box2[0])
                y1 = max(box1[1], box2[1])
                x2 = min(box1[2], box2[2])
                y2 = min(box1[3], box2[3])
                inter_area = max(0, x2 - x1) * max(0, y2 - y1)
                box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
                box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
                return inter_area / float(box1_area + box2_area - inter_area) if inter_area > 0 else 0

            # Assign text to the correct cell in the matrix
            for i in range(len(horiz_lines)):
                for j in range(len(vert_lines)):
                    target_box = [vert_boxes[vert_lines[ordered_boxes[j]]][0], horiz_boxes[horiz_lines[i]][1],
                                  vert_boxes[vert_lines[ordered_boxes[j]]][2], horiz_boxes[horiz_lines[i]][3]]
                    for b in range(len(boxes)):
                        detected_box = [boxes[b][0][0], boxes[b][0][1], boxes[b][2][0], boxes[b][2][1]]
                        if iou(target_box, detected_box) > 0.2:
                            out_array[i][j] = texts[b]

            # Convert the list to a DataFrame
            df = pd.DataFrame(out_array)

            # Remove empty columns and rows
            df = df.loc[:, (df != "").any(axis=0)]  # Remove empty columns
            df = df.loc[(df != "").any(axis=1), :]  # Remove empty rows


            # Save to CSV
            output_csv_path = os.path.join(output_directory, f"{filename.split('.')[0]}.csv")
            df.to_csv(output_csv_path, index=False, header=False)
            print(f"✅ Data saved to {output_csv_path}")

# Specify input and output directories
directory_path = "/content/pages"  # Folder containing cropped images
output_directory = "/content/output_csvs"  # Folder to save CSVs

# Run the function
process_images_and_generate_csv(directory_path, output_directory)


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:00<00:00, 4741.64it/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10000/10000 [00:01<00:00, 9952.29it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:00<00:00, 2975.40it/s]

[2025/02/01 04:47:09] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='CRNN', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dic

[2025/02/01 04:47:10] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2025/02/01 04:47:11] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2025/02/01 04:47:12] ppocr DEBUG: dt_boxes num : 278, elapsed : 0.38512730598449707
[2025/02/01 04:47:12] ppocr DEBUG: rec_res num  : 278, elapsed : 0.9036765098571777
✅ Data saved to /content/output_csvs/cropped_page6_table.csv
[2025/02/01 04:47:13] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2025/02/01 04:47:13] ppocr DEBUG: dt_boxes num : 22, elapsed : 0.06377887725830078
[2025/02/01 04:47:13] ppocr DEBUG: rec_res num  : 22, elapsed : 0.05519723892211914
✅ Data saved to /content/output_csvs/cropped_page7_table.csv
[2025/02/01 04:47:13] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2025/02/01 04:47:13] ppocr DEBUG: dt_boxes num : 14,

# Conveting the csv files into one sinple txt file

In [6]:
import os
import csv
import re

def csvs_to_single_text(input_directory, output_text_file):
    """
    Merge all CSV files in a directory into a single text file.
    - Skips the first column.
    - Skips empty rows.
    - Ensures no consecutive commas appear.
    """

    # Get all CSV files in the directory
    csv_files = [f for f in os.listdir(input_directory) if f.endswith('.csv')]

    if not csv_files:
        print("No CSV files found in the directory.")
        return

    with open(output_text_file, mode='w', encoding='utf-8') as out_file:
        for file in sorted(csv_files):  # Sort for consistency
            csv_file_path = os.path.join(input_directory, file)

            with open(csv_file_path, mode='r', encoding='utf-8-sig') as csv_file:
                reader = csv.reader(csv_file)

                next(reader, None)  # Skip the first row (header)

                for row in reader:
                    if len(row) > 1:  # Ensure valid row exists
                        cleaned_row = [value.strip() for value in row[1:] if value.strip()]  # Remove empty columns
                        if cleaned_row:
                            line = "  ".join(cleaned_row)
                            line = re.sub(r'\s*,\s*', ' , ', line)  # Remove extra spaces around commas
                            out_file.write(line + "\n")

            out_file.write("\n" + "-" * 50 + "\n")  # Separator for clarity

    print(f"✅ Merged all CSVs into {output_text_file}")

# 🔹 Usage
input_directory = "/content/output_csvs"  # Update with your CSV directory path
output_text_file = "output.txt"  # Final merged output file
csvs_to_single_text(input_directory, output_text_file)


✅ Merged all CSVs into output.txt


In [7]:
!pip install langflow  # Install langflow if it's not already installed

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mcp to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of crewai-tools to determine which version is compatible with other requirements. This could take a while.
  Preparing meta

# **Automated Financial Data Extraction Using Langflow API**

In [9]:
import os
import json
import argparse
import requests
from argparse import RawTextHelpFormatter
from typing import Optional
import warnings

try:
    from langflow.load import upload_file
except ImportError:
    warnings.warn("Langflow provides a function to help you upload files to the flow. Please install langflow to use it.")
    upload_file = None

BASE_API_URL = "https://api.langflow.astra.datastax.com"
LANGFLOW_ID = "c6d524b3-2ff0-4e73-b765-449e37ffe560"
FLOW_ID = "8327ee1c-0964-4c74-8a5c-73948095f523"
APPLICATION_TOKEN = "AstraCS:jGUxqLLrjRBRMFZIGizOhEoA:d473d5249f470a4b6589ad16930b9f215589465d73ecdaa3ab4b70e86fb55cda"
ENDPOINT = ""  # Set a specific endpoint if needed

TWEAKS = {
    "Prompt-byv20": {},
    "ChatOutput-QPTF5": {},
    "ChatInput-Cljjm": {},
    "GroqModel-hTdjo": {}
}

def run_flow(message: str, endpoint: str, output_type: str = "chat", input_type: str = "chat",
             tweaks: Optional[dict] = None, application_token: Optional[str] = None) -> dict:
    """
    Run a flow with a given message and optional tweaks.

    :param message: The message to send to the flow
    :param endpoint: The ID or the endpoint name of the flow
    :param tweaks: Optional tweaks to customize the flow
    :return: The JSON response from the flow
    """
    api_url = f"{BASE_API_URL}/lf/{LANGFLOW_ID}/api/v1/run/{endpoint}"

    payload = {
        "input_value": message,
        "output_type": output_type,
        "input_type": input_type,
    }
    headers = None
    if tweaks:
        payload["tweaks"] = tweaks
    if application_token:
        headers = {"Authorization": "Bearer " + application_token, "Content-Type": "application/json"}
    response = requests.post(api_url, json=payload, headers=headers)
    return response.json()

def main():
    # Instead of parsing command-line arguments, define variables directly
    output_file = "output.txt"  # Path to the text file
    endpoint = ENDPOINT or FLOW_ID  # Flow ID or endpoint
    tweaks = TWEAKS  # Tweaks for the flow
    application_token = APPLICATION_TOKEN  # Authentication token
    output_type = "chat"  # Output type
    input_type = "chat"  # Input type
    upload_file_path = None  # Path to file to upload
    components = None  # Components for upload

    if not os.path.exists(output_file):
        raise FileNotFoundError(f"❌ The file '{output_file}' does not exist. Ensure the CSV processing step has been completed.")

    with open(output_file, "r", encoding="utf-8") as file:
        message = file.read().strip()


    if upload_file_path:
        if not upload_file:
            raise ImportError("Langflow is not installed. Please install it to use the upload_file function.")
        elif not components:
            raise ValueError("You need to provide the components to upload the file to.")
        tweaks = upload_file(file_path=upload_file_path, host=BASE_API_URL, flow_id=ENDPOINT, components=components, tweaks=tweaks)

    response = run_flow(
        message=message,
        endpoint=endpoint,
        output_type=output_type,
        input_type=input_type,
        tweaks=tweaks,
        application_token=application_token
    )

    #print(json.dumps(response, indent=2))
    print(response['outputs'][0]['outputs'][0]['results']['message']['data']['text'])

if __name__ == "__main__":
      main()


Based on the provided financial statements, I will extract the key financial metrics for the latest quarter (December 31, 2024) from the "Unaudited Consolidated Statement of Profit and Loss" table.

**Revenue/Sales:**
```json
{
  "Revenue": 417.58,
  "Total Revenue": 417.58,
  "Sales Revenue": 375.39,
  "Net Sales": 375.39,
  "Gross Sales": 375.39,
  "Turnover": 417.58,
  "Top-line Revenue": 417.58,
  "Total Sales": 417.58,
  "Operating Revenue": 417.58
}
```

**Operating Profit (EBIT/Profit Before Tax):**
```json
{
  "Operating Profit": 221.19,
  "Operating Income": 221.19,
  "Earnings Before Interest & Taxes (EBIT)": 221.19,
  "Profit from Operations": 221.19,
  "Operating Earnings": 221.19,
  "Core Business Profit": 221.19,
  "Income from Continuing Operations": 221.19,
  "EBITDA (Earnings Before Interest, Taxes, Depreciation, and Amortization)": 221.19
}
```

**Net Profit (PAT/Profit After Tax):**
```json
{
  "Net Profit": 150.69,
  "Net Income": 150.69,
  "Bottom-line Profit": 150